In [4]:

"""
F311024 December 2023
This program utilizes Jupyter widgets to create an interactive game management system with features for game search, rental, return, feedback, and inventory pruning.

Functions:
1. `give_feedback(game_id, rating, comment)`: Prompts the user to give feedback on a game, including a rating and comment. Returns True if feedback is successfully collected, False otherwise.

2. `search_games(term, search_option)`: Searches for games based on the specified term and search option (title, genre, or platform). Returns a list of game search results.

3. `display_search_results(results)`: Displays the search results using Jupyter widgets.

4. `handle_search_button_click(b)`: Event handler for the search button. Invokes the search function and displays the results.

5. `handle_clear_button_click(b)`: Event handler for the clear button. Clears the search results.

6. `handle_rent_game_button_click(b)`: Event handler for the rent game button. Handles the game rental process, checking customer existence and updating rental information.

7. `handle_return_game_button_click(b)`: Event handler for the return game button. Handles the game return process, checking game existence and updating return information.

8. `handle_enter_feedback_button_click(b)`: Event handler for the enter feedback button. Handles user feedback entry.

9. `handle_inventory_pruning_button_click(b)`: Event handler for the inventory pruning button. Calls the inventory pruning function and displays three graphs (average rental duration, rental frequency, average rating).

10. `main()`: The main function that orchestrates the interactive game management system.

"""

import matplotlib.pyplot as plt
import ipywidgets as widgets
import database
import gamesearch
import gamerent
import gamereturn
import csv
%matplotlib inline
import inventorypruning
import subscriptionManager
import feedbackManager


def give_feedback(game_id, rating, comment):
    # Parameters:
    #   - game_ID (str): The ID of the game to give feedback on.
    # Description: Prompts the user to give feedback on a game, including a rating and comment.
    success = False
    feedbackManager.add_feedback(game_id, rating, comment, 'Game_Feedback.txt')
    print("Thank you")
    success = True

    return success


def search_games(term, search_option):
    games = database.read_game_data()
    results = []

    if search_option == 1:
        results = gamesearch.search_game_by_title(games, term)
    elif search_option == 2:
        results = gamesearch.search_game_by_genre(games, term)
    elif search_option == 3:
        results = gamesearch.search_game_by_platform(games, term)

    return results

def display_search_results(results):
    output_box.clear_output(wait=True)

    with output_box:
        for result in results:
            gamesearch.print_results(result)

game_search_box = widgets.Text(description="Search:")
search_option_dropdown = widgets.Dropdown(
    options=[("Title", 1), ("Genre", 2), ("Platform", 3)],
    value=1,
    description="Search by:",
)
search_button = widgets.Button(description="Search")
clear_button = widgets.Button(description="Clear")
output_box = widgets.Output()

def handle_search_button_click(b):
    term = game_search_box.value
    search_option = search_option_dropdown.value
    results = search_games(term, search_option)
    display_search_results(results)

def handle_clear_button_click(b):
    output_box.clear_output(wait=True)

search_button.on_click(handle_search_button_click)
clear_button.on_click(handle_clear_button_click)


# Arrange widgets in a VBox for the Game Search Window
game_search_window = widgets.VBox([
    widgets.Label("Game Search"),
    game_search_box,
    search_option_dropdown,
    search_button,
    clear_button,
    output_box
])

display(game_search_window)

#gamerent window
customer_id_input = widgets.Text(description="Customer ID:")
game_id_input = widgets.Text(description="Game ID:")
rental_date_input = widgets.Text(description="Rental Date:")
rent_game_button = widgets.Button(description="Rent Game")
rent_game_output = widgets.Output()

def handle_rent_game_button_click(b):
    
    with rent_game_output:
        rent_game_output.clear_output(wait=True)
        
        print("Button clicked!")

        customer_id = customer_id_input.value
        game_id = game_id_input.value
        rental_date = rental_date_input.value
        
        print(f"Handling rent game button click. Customer ID: {customer_id}, Game ID: {game_id}, Rental Date: {rental_date}")

        if gamerent.customer_exists(customer_id):
            print("works")
            print(gamerent.rentgame2(game_id, customer_id, rental_date))
            #print("rental.txt updated:")  
            #with open('rental.txt','r') as f:
                #print(f.read())
        else:
            print("Customer does not exist")

rent_game_button.on_click(handle_rent_game_button_click)

# Arrange widgets in a VBox for the Game Rent Window
game_rent_window = widgets.VBox([
    widgets.Label("Game Rent"),
    customer_id_input,
    game_id_input,
    rental_date_input,
    rent_game_button,
    rent_game_output
])

display(game_rent_window)

#returngame window 
# Widgets for the Game Return Window
game_return_id_input = widgets.Text(description="Game ID:")
return_date_input = widgets.Text(description="Return Date:")
return_game_button = widgets.Button(description="Return Game")
feedback_rating_input = widgets.BoundedIntText(value=0, min=0, max=5, description="Feedback Rating:")
feedback_comment_input = widgets.Text(description="Feedback Comment:")
enter_feedback_button = widgets.Button(description="Enter Feedback")
return_game_output = widgets.Output()
rental_file_contents = widgets.Text()

def handle_return_game_button_click(b):
    with return_game_output:
        return_game_output.clear_output(wait=True)
        
        print("Button clicked!")

        game_id = game_return_id_input.value
        return_date = return_date_input.value
        
        print(f"Handling return game button click.  Game ID: {game_id}, Return Date: {return_date}")

        if database.game_exists(game_id):
            print("works")
            if not gamereturn.already_returned(game_id):
                print("Game successfully returned.")
                gamereturn.return_game2(game_id, return_date)
                print("Game successfully returned.")
            else:
                print("Game is already returned.")
        else:
            print("Game ID does not exist.")

return_game_button.on_click(handle_return_game_button_click)

def handle_enter_feedback_button_click(b):
    with return_game_output:
        return_game_output.clear_output(wait=True)
        
        print("Button clicked!")

        game_id = game_return_id_input.value
        rating = feedback_rating_input.value
        comment = feedback_comment_input.value
        
        print(f"Handling feedback button click.  Game ID: {game_id}, Rating: {rating}, Comment: {comment}")
        
        
        print("works")
        give_feedback(game_id, rating, comment)

feedback_rating_input.disabled = False
feedback_comment_input.disabled = False
enter_feedback_button.disabled = False
enter_feedback_button.on_click(handle_enter_feedback_button_click)

# Arrange widgets in a VBox for the Game Return Window
game_return_window = widgets.VBox([
    widgets.Label("Game Return"),
    game_return_id_input,
    return_date_input,
    return_game_button,
    feedback_rating_input,
    feedback_comment_input,
    enter_feedback_button,
    return_game_output
])

# Display the Game Return Window
display(game_return_window)

#inventory pruning
inventory_pruning_button = widgets.Button(description="Inventory Pruning")
inventory_pruning_output = widgets.Output()

def handle_inventory_pruning_button_click(b):
    with inventory_pruning_output:
        inventory_pruning_output.clear_output(wait=True)

        # Call the inventory pruning function
        fig1, fig2, fig3 = inventorypruning.inventory_pruning()
        
        plt.bar(fig1.keys(), fig1.values())
        plt.xlabel('Game ID')
        plt.ylabel('Average Rental Duration')
        plt.title('Average Rental Duration of Each Game')
        plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
        plt.show()

        plt.bar(fig2.keys(), fig2.values())
        plt.xlabel('Game ID')
        plt.ylabel('Rental Frequency')
        plt.title('Rental Frequency of Each Game')
        plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
        plt.show()

        plt.bar(fig3.keys(), fig3.values())
        plt.xlabel('Game ID')
        plt.ylabel('Average Rating')
        plt.title('Average Rating of Each Game')
        plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
        plt.show()

        plt.tight_layout()


inventory_pruning_button.on_click(handle_inventory_pruning_button_click)

# Arrange widgets in a VBox for the Inventory Pruning Window
inventory_pruning_window = widgets.VBox([
    widgets.Label("Inventory Pruning"),
    inventory_pruning_button,
    inventory_pruning_output
])

# Display the Inventory Pruning Window
display(inventory_pruning_window)



<Figure size 640x480 with 0 Axes>